In [79]:
%pip install datasets
%pip install transformers
%pip install peft
%pip install evaluate
%pip install scikit-learn



from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from turtle import pd
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('Joanne/Metaphors_and_Analogies','Pairs_Jankowiac_random_split')
dataset

Using the latest cached version of the module from C:\Users\zhuocheng\.cache\huggingface\modules\datasets_modules\datasets\Joanne--Metaphors_and_Analogies\beea150ea6a22d819d08cdd41e16eca5d8f32c108cc19640e8a32d10cbe8579f (last modified on Mon Apr 29 17:11:06 2024) since it couldn't be found locally at Joanne/Metaphors_and_Analogies, or remotely on the Hugging Face Hub.


DatasetDict({
    train: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
        num_rows: 36
    })
    test: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
        num_rows: 144
    })
})

In [81]:
# Get the first row of the dataset
first_row = dataset['train']

# Print the first row
print(first_row)

Dataset({
    features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
    num_rows: 180
})


In [82]:
# assume 'dataset' is your dataset object
label_array = np.array(dataset['train']['label'])

label_dtype = label_array.dtype

label_array

array([2, 1, 2, 1, 1, 0, 0, 2, 1, 1, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 2, 2,
       2, 1, 0, 0, 0, 0, 2, 2, 0, 1, 0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 0, 2,
       0, 1, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 1, 2, 1, 1, 2, 2,
       1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 2, 2, 0, 0, 0,
       0, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 0, 2,
       2, 1, 1, 2, 2, 1, 0, 2, 2, 1, 0, 1, 1, 0, 2, 1, 0, 1, 0, 0, 0, 1,
       0, 2, 1, 0, 2, 0, 1, 0, 0, 1, 2, 1, 2, 0, 0, 0, 1, 0, 0, 2, 2, 0,
       2, 0, 0, 2])

In [83]:
model_checkpoint = 'distilbert-base-uncased'
# define label maps
id2label = {0: "Anomaly", 1: "literal", 2 : "metaphor"}
label2id = {"Anomaly":0, "literal":1, "metaphor":2}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [84]:
# create tokenize function

# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


def tokenize_function(examples):
    # extract text
    text = examples["sentence"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 36/36 [00:00<00:00, 6687.11 examples/s]


DatasetDict({
    train: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds', 'input_ids', 'attention_mask'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds', 'input_ids', 'attention_mask'],
        num_rows: 36
    })
    test: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds', 'input_ids', 'attention_mask'],
        num_rows: 144
    })
})

In [85]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [86]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [87]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Anomaly
Not a fan, don't recommed. - Anomaly
Better than the first one. - Anomaly
This is not worth watching even once. - Anomaly
This one is a pass. - Anomaly


In [88]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [89]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [90]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 629,763 || all params: 67,585,542 || trainable%: 0.9318013607111414


In [91]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [93]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [96]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

In [97]:
# train model
trainer.train()

 10%|█         | 45/450 [00:03<00:22, 17.91it/s]c:\Users\zhuocheng\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\utils\other.py:581: UserWarning: Unable to fetch remote file due to the following error (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 18f8bd39-1f9e-4dbb-aba8-7812e147575a)') - silently ignoring the lookup for the file config.json in distilbert-base-uncased.
  warnings.warn(
c:\Users\zhuocheng\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in distilbert-base-uncased - will assume that the vocabulary was not modified.
  warnings.warn(
 10%|█         | 46/450 [00:03<01:24,  4.77it/s]

{'eval_loss': 0.5115641951560974, 'eval_accuracy': {'accuracy': 0.8194444444444444}, 'eval_runtime': 0.9455, 'eval_samples_per_second': 152.305, 'eval_steps_per_second': 38.076, 'epoch': 1.0}


 20%|██        | 90/450 [00:07<00:22, 15.68it/s]

{'eval_loss': 0.5675480365753174, 'eval_accuracy': {'accuracy': 0.8472222222222222}, 'eval_runtime': 0.9378, 'eval_samples_per_second': 153.556, 'eval_steps_per_second': 38.389, 'epoch': 2.0}


 30%|███       | 135/450 [00:11<00:19, 16.39it/s]

...

100%|██████████| 450/450 [00:41<00:00, 17.22it/s]

{'eval_loss': 0.8741100430488586, 'eval_accuracy': {'accuracy': 0.8541666666666666}, 'eval_runtime': 0.9294, 'eval_samples_per_second': 154.939, 'eval_steps_per_second': 38.735, 'epoch': 10.0}


100%|██████████| 450/450 [00:41<00:00, 10.75it/s]

{'train_runtime': 41.86, 'train_samples_per_second': 43.0, 'train_steps_per_second': 10.75, 'train_loss': 0.10120034959581163, 'epoch': 10.0}


TrainOutput(global_step=450, training_loss=0.10120034959581163, metrics={'train_runtime': 41.86, 'train_samples_per_second': 43.0, 'train_steps_per_second': 10.75, 'total_flos': 3656219502240.0, 'train_loss': 0.10120034959581163, 'epoch': 10.0})